In [1]:
import pyspark

In [2]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
sc=SparkContext('local','Spark SQL')
sqlc=SQLContext(sc)

## agg

In [3]:
#aggregation
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3)],['from','to','amt'])
y=x.agg({'amt':'avg'})
x.show()
y.show()

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
+-----+-----+---+

+-------------------+
|           avg(amt)|
+-------------------+
|0.20000000000000004|
+-------------------+



## alias

In [4]:
#alias
from pyspark.sql.functions import col
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3)],['from','to','amt'])
y=x.alias('transactions')
x.show()
y.show()
y.select(col('transactions.to')).show()

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
+-----+-----+---+

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
+-----+-----+---+

+-----+
|   to|
+-----+
|  Bob|
|  kev|
|david|
+-----+



## cache

In [5]:
#cache
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3)],['from','to','amt'])
x.cache()
print(x.count())
print(x.count())

3
3


## coalesce

In [6]:
#coalesce
x_rdd=sc.parallelize([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3)],2)
x=sqlc.createDataFrame(x_rdd,['from','to','amt'])
y=x.coalesce(numPartitions=1)
print(x.rdd.getNumPartitions())
print(y.rdd.getNumPartitions())

2
1


## collect

In [7]:
#collect
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3)],['from','to','amt'])
x.rdd.collect()


[Row(from='Alice', to='Bob', amt=0.1),
 Row(from='fred', to='kev', amt=0.2),
 Row(from='nico', to='david', amt=0.3)]

In [8]:
y=x.collect() #creates list on driver
print(y)

[Row(from='Alice', to='Bob', amt=0.1), Row(from='fred', to='kev', amt=0.2), Row(from='nico', to='david', amt=0.3)]


## columns

In [9]:
#columns
x=x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3)],['from','to','amt'])
y=x.columns
print(y)


['from', 'to', 'amt']


## corr

In [10]:
#correlation (corr)
x=sqlc.createDataFrame([('Alice','Bob',0.1,0.001),('fred','kev',0.2,0.005),('nico','david',0.3,0.1)],
                         ['from','to','amt','fee'])
y=x.corr('amt','fee')
x.show()
print(y)



+-----+-----+---+-----+
| from|   to|amt|  fee|
+-----+-----+---+-----+
|Alice|  Bob|0.1|0.001|
| fred|  kev|0.2|0.005|
| nico|david|0.3|  0.1|
+-----+-----+---+-----+

0.8833184961559478


## count

In [11]:
#count
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3)],['from','to','amt'])
print(x.count())

3


## cov

In [12]:
#covariance
x=sqlc.createDataFrame([('Alice','Bob',0.1,0.05),('fred','kev',0.2,0.4),('nico','david',0.3,0.8)],
                       ['from','to','amt','fee'])
y=x.cov('amt','fee')
x.show()
print(y)

+-----+-----+---+----+
| from|   to|amt| fee|
+-----+-----+---+----+
|Alice|  Bob|0.1|0.05|
| fred|  kev|0.2| 0.4|
| nico|david|0.3| 0.8|
+-----+-----+---+----+

0.03749999999999999


## crosstab

In [13]:
#crosstab
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3)],['from','to','amt'])
y=x.crosstab('from','to')
x.show()
y.show()

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
+-----+-----+---+

+-------+---+-----+---+
|from_to|Bob|david|kev|
+-------+---+-----+---+
|   fred|  0|    0|  1|
|   nico|  0|    1|  0|
|  Alice|  1|    0|  0|
+-------+---+-----+---+



## cube

In [14]:
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3)],['from','to','amt'])
y=x.cube('from','to')
x.show()
print(y) #grouped data object,aggregations will be applied to numeric columns

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
+-----+-----+---+



In [15]:
y.sum().show()

+-----+-----+------------------+
| from|   to|          sum(amt)|
+-----+-----+------------------+
| fred| null|               0.2|
| null| null|0.6000000000000001|
| null|  Bob|               0.1|
| null|david|               0.3|
| fred|  kev|               0.2|
| nico|david|               0.3|
| nico| null|               0.3|
|Alice|  Bob|               0.1|
| null|  kev|               0.2|
|Alice| null|               0.1|
+-----+-----+------------------+



In [16]:
y.max().show()

+-----+-----+--------+
| from|   to|max(amt)|
+-----+-----+--------+
| fred| null|     0.2|
| null| null|     0.3|
| null|  Bob|     0.1|
| null|david|     0.3|
| fred|  kev|     0.2|
| nico|david|     0.3|
| nico| null|     0.3|
|Alice|  Bob|     0.1|
| null|  kev|     0.2|
|Alice| null|     0.1|
+-----+-----+--------+



## describe

In [17]:
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3)],['from','to','amt'])
x.describe().show()
x.show()

+-------+-----+----+-------------------+
|summary| from|  to|                amt|
+-------+-----+----+-------------------+
|  count|    3|   3|                  3|
|   mean| null|null|0.20000000000000004|
| stddev| null|null|0.09999999999999998|
|    min|Alice| Bob|                0.1|
|    max| nico| kev|                0.3|
+-------+-----+----+-------------------+

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
+-----+-----+---+



## distinct

In [18]:
#distinct
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3),('fred','kev',0.2)],
                       ['from','to','amt'])
y=x.distinct()
x.show()
y.show()

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
| fred|  kev|0.2|
+-----+-----+---+

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
| nico|david|0.3|
|Alice|  Bob|0.1|
| fred|  kev|0.2|
+-----+-----+---+



## drop

In [19]:
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3)],['from','to','amt'])
y=x.drop('amt')
x.show()
y.show()

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
+-----+-----+---+

+-----+-----+
| from|   to|
+-----+-----+
|Alice|  Bob|
| fred|  kev|
| nico|david|
+-----+-----+



## dropDuplicates

In [20]:
#dropDuplicates
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3),('nico','david',0.8),('fred','kev',0.2)],
                       ['from','to','amt'])
y=x.dropDuplicates(subset=['from','to'])
x.show()
y.show()

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
| nico|david|0.8|
| fred|  kev|0.2|
+-----+-----+---+

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
| nico|david|0.3|
|Alice|  Bob|0.1|
| fred|  kev|0.2|
+-----+-----+---+



## dropna

In [21]:
#dropna
x=sqlc.createDataFrame([(None,'Bob',0.1),('fred',None,0.2),('nico','david',None),('nico','vinny',0.5)],['from','to','amt'])
y=x.dropna(subset=['from','to'])
x.show()
y.show()

+----+-----+----+
|from|   to| amt|
+----+-----+----+
|null|  Bob| 0.1|
|fred| null| 0.2|
|nico|david|null|
|nico|vinny| 0.5|
+----+-----+----+

+----+-----+----+
|from|   to| amt|
+----+-----+----+
|nico|david|null|
|nico|vinny| 0.5|
+----+-----+----+



## dtypes

In [22]:
#dtypes
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3)],['from','to','amt'])
y=x.dtypes
x.show()
print(y)

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
+-----+-----+---+

[('from', 'string'), ('to', 'string'), ('amt', 'double')]


In [23]:
type(y)

list

## explain

In [24]:
#explain
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3)],['from','to','amt'])
y=x.explain()
x.show()
x.agg({'amt':'avg'}).explain(extended=True)

== Physical Plan ==
Scan ExistingRDD[from#687,to#688,amt#689]
+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
+-----+-----+---+

== Parsed Logical Plan ==
'Aggregate ['avg(amt#689) AS avg(amt)#706]
+- LogicalRDD [from#687, to#688, amt#689], false

== Analyzed Logical Plan ==
avg(amt): double
Aggregate [avg(amt#689) AS avg(amt)#706]
+- LogicalRDD [from#687, to#688, amt#689], false

== Optimized Logical Plan ==
Aggregate [avg(amt#689) AS avg(amt)#706]
+- Project [amt#689]
   +- LogicalRDD [from#687, to#688, amt#689], false

== Physical Plan ==
*(2) HashAggregate(keys=[], functions=[avg(amt#689)], output=[avg(amt)#706])
+- Exchange SinglePartition
   +- *(1) HashAggregate(keys=[], functions=[partial_avg(amt#689)], output=[sum#711, count#712L])
      +- *(1) Project [amt#689]
         +- Scan ExistingRDD[from#687,to#688,amt#689]


## fillna

In [25]:
#fillna
x=sqlc.createDataFrame([('Alice',None,0.1),('fred','kev',0.2),(None,'david',0.3)],['from','to','amt'])
y=x.fillna(value='unknown',subset=['from','to'])
x.show()
y.show()

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice| null|0.1|
| fred|  kev|0.2|
| null|david|0.3|
+-----+-----+---+

+-------+-------+---+
|   from|     to|amt|
+-------+-------+---+
|  Alice|unknown|0.1|
|   fred|    kev|0.2|
|unknown|  david|0.3|
+-------+-------+---+



## filter

In [26]:
#filter
x=sqlc.createDataFrame([('Alice',None,0.1),('fred','kev',0.2),(None,'david',0.3)],['from','to','amt'])
y=x.filter('amt > 0.1')
x.show()
y.show()

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice| null|0.1|
| fred|  kev|0.2|
| null|david|0.3|
+-----+-----+---+

+----+-----+---+
|from|   to|amt|
+----+-----+---+
|fred|  kev|0.2|
|null|david|0.3|
+----+-----+---+



## first

In [27]:
#first
x=sqlc.createDataFrame([('Alice',None,0.1),('fred','kev',0.2),(None,'david',0.3)],['from','to','amt'])
y=x.first()
x.show()
print(y)

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice| null|0.1|
| fred|  kev|0.2|
| null|david|0.3|
+-----+-----+---+

Row(from='Alice', to=None, amt=0.1)


## flatMap

In [28]:
#flatMap
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3)],['from','to','amt'])
y=x.rdd.flatMap(lambda q:(q[0],q[2])) #dataframe don't have flatmap attribute,so change it to rdd(x.rdd.flatMap)
x.show()
print(y)
y.collect()

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
+-----+-----+---+

PythonRDD[306] at RDD at PythonRDD.scala:53


['Alice', 0.1, 'fred', 0.2, 'nico', 0.3]

## foreach

In [29]:
#foreach

fn='foreachExample.txt'
open(fn,'w').close() #clear the file

def append(el,f):
    print(el,file=open(f,'a+'))

#example
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3)],['from','to','amt'])
y=x.foreach(lambda x:append(x,fn)) #writes into the file
x.show()
print(y)

with open(fn,'r') as foreachExample:
    print(foreachExample.read())

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
+-----+-----+---+

None
Row(from='Alice', to='Bob', amt=0.1)
Row(from='fred', to='kev', amt=0.2)
Row(from='nico', to='david', amt=0.3)



## foreachPartition

In [30]:
#foreachPartition
fn='foreachPartitionExample.txt'
open(fn,'w').close() #clear the file

def foreachpartition(partition,f):
    print([el for el in partition],file=open(f,'a+'))

x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3)],['from','to','amt'])
x=x.repartition(2) #partitioning into 2
y=x.foreachPartition(lambda x:foreachpartition(x,fn))
x.show()
print(y)
with open(fn,'r') as foreachExample:
    print(foreachExample.read())
    

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
+-----+-----+---+

None
[Row(from='Alice', to='Bob', amt=0.1), Row(from='fred', to='kev', amt=0.2)]
[Row(from='nico', to='david', amt=0.3)]



## freqItems

In [31]:
#freqItems
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3),
                        ('Alice','Olive',0.1)],['from','to','amt'])
y=x.freqItems(cols=['from','amt'],support=0.8)
x.show()
y.show()

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
|Alice|Olive|0.1|
+-----+-----+---+

+--------------+-------------+
|from_freqItems|amt_freqItems|
+--------------+-------------+
|            []|           []|
+--------------+-------------+



## groupBy

In [32]:
#groupby
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3)],['from','to','amt'])
y=x.groupBy(['from'])
x.show()
print(type(y))

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
+-----+-----+---+

<class 'pyspark.sql.group.GroupedData'>


In [33]:
#groupby(col1).avg(col2)
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3),('Alice','Kun',0.5)],['from','to','amt'])
y=x.groupBy(['from']).avg('amt')
x.show()
print(type(y))
y.show()

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
|Alice|  Kun|0.5|
+-----+-----+---+

<class 'pyspark.sql.dataframe.DataFrame'>
+-----+--------+
| from|avg(amt)|
+-----+--------+
| fred|     0.2|
|Alice|     0.3|
| nico|     0.3|
+-----+--------+



## head

In [34]:
#head
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3)],['from','to','amt'])
y=x.head(2)
x.show()
print(y) #y is a list, not a dataframe

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
+-----+-----+---+

[Row(from='Alice', to='Bob', amt=0.1), Row(from='fred', to='kev', amt=0.2)]


## intersect

In [35]:
#intersect
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3)],['from','to','amt'])
y=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','Kun',0.2),('nico','david',0.2)],['from','to','amt'])
x.show()
y.show()
z=x.intersect(y)
z.show()

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
+-----+-----+---+

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  Kun|0.2|
| nico|david|0.2|
+-----+-----+---+

+-----+---+---+
| from| to|amt|
+-----+---+---+
|Alice|Bob|0.1|
+-----+---+---+



## isLocal

In [36]:
#isLocal
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3)],['from','to','amt'])
y=x.isLocal()
x.show()
print(y) #y is object on driver

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
+-----+-----+---+

False


## join

In [37]:
#join(inner)
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3)],['from','to','amt'])
y=sqlc.createDataFrame([('Alice',20),('Kun',25),('david',32),('kev',22)],['name','age'])
z=x.join(y,x.to==y.name,'inner').select('from','to','amt','age')
x.show()
y.show()
z.show()

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
+-----+-----+---+

+-----+---+
| name|age|
+-----+---+
|Alice| 20|
|  Kun| 25|
|david| 32|
|  kev| 22|
+-----+---+

+----+-----+---+---+
|from|   to|amt|age|
+----+-----+---+---+
|nico|david|0.3| 32|
|fred|  kev|0.2| 22|
+----+-----+---+---+



## limit

In [40]:
#join(inner)
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3)],['from','to','amt'])
y=x.limit(2)
x.show()
y.show()

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
+-----+-----+---+

+-----+---+---+
| from| to|amt|
+-----+---+---+
|Alice|Bob|0.1|
| fred|kev|0.2|
+-----+---+---+



## map

In [41]:
#map
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3)],['from','to','amt'])
y=x.rdd.map(lambda x:x.amt+1)
x.show()
y.collect()

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
+-----+-----+---+



[1.1, 1.2, 1.3]

## mapPartitions

In [46]:
#mapPartitions
def amt_sum(partition):
    yield sum([el.amt for el in partition])
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3)],['from','to','amt'])
x=x.repartition(2)
y=x.rdd.mapPartitions(lambda p:amt_sum(p))
x.show()
print(x.rdd.glom().collect()) #glom() transforms each partition into tuples(immutable) of rdd
print(y.collect())
print(y.glom().collect())

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
+-----+-----+---+

[[Row(from='Alice', to='Bob', amt=0.1), Row(from='fred', to='kev', amt=0.2)], [Row(from='nico', to='david', amt=0.3)]]
[0.30000000000000004, 0.3]
[[0.30000000000000004], [0.3]]


## na

In [49]:
#na(supports fill,drop,replace methods)
x=sqlc.createDataFrame([(None,'Bob',0.1),('fred',None,0.2),('nico','david',None),('kev','kun',0.9)],['from','to','amt'])
y=x.na #supports drop, fill, replace
x.show()
print(y)
y.drop().show()
y.fill({'from':'unknown','to':'unknown','amt':0.0}).show()
y.fill(0).show()

+----+-----+----+
|from|   to| amt|
+----+-----+----+
|null|  Bob| 0.1|
|fred| null| 0.2|
|nico|david|null|
| kev|  kun| 0.9|
+----+-----+----+

+----+---+---+
|from| to|amt|
+----+---+---+
| kev|kun|0.9|
+----+---+---+

+-------+-------+---+
|   from|     to|amt|
+-------+-------+---+
|unknown|    Bob|0.1|
|   fred|unknown|0.2|
|   nico|  david|0.0|
|    kev|    kun|0.9|
+-------+-------+---+

+----+-----+---+
|from|   to|amt|
+----+-----+---+
|null|  Bob|0.1|
|fred| null|0.2|
|nico|david|0.0|
| kev|  kun|0.9|
+----+-----+---+



## orderBy

In [55]:
#orderBy
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3),('Alice','Bob',0.4)],['from','to','amt'])
y=x.orderBy(['from','amt'],ascending=[True,False])
x.show()
y.show()

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
|Alice|  Bob|0.4|
+-----+-----+---+

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.4|
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
+-----+-----+---+



## printSchema

In [58]:
#printSchema
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3),('Alice','Bob',0.4)],
                       ['from','to','amt'])
x.printSchema()
x.show()


root
 |-- from: string (nullable = true)
 |-- to: string (nullable = true)
 |-- amt: double (nullable = true)

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
|Alice|  Bob|0.4|
+-----+-----+---+



## randomSplit

In [72]:
#randomSplit
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3),('Alice','Bob',0.4)],
                       ['from','to','amt'])
y=x.randomSplit([0.5,0.2])
x.show()
print(y)
y[0].show()
y[1].show()

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
|Alice|  Bob|0.4|
+-----+-----+---+

[DataFrame[from: string, to: string, amt: double], DataFrame[from: string, to: string, amt: double]]
+-----+---+---+
| from| to|amt|
+-----+---+---+
|Alice|Bob|0.1|
|Alice|Bob|0.4|
| fred|kev|0.2|
+-----+---+---+

+----+-----+---+
|from|   to|amt|
+----+-----+---+
|nico|david|0.3|
+----+-----+---+



## rdd

In [73]:
#rdd
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3),('Alice','Bob',0.4)],
                       ['from','to','amt'])
y=x.rdd
x.show()
y.collect()

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
|Alice|  Bob|0.4|
+-----+-----+---+



[Row(from='Alice', to='Bob', amt=0.1),
 Row(from='fred', to='kev', amt=0.2),
 Row(from='nico', to='david', amt=0.3),
 Row(from='Alice', to='Bob', amt=0.4)]

## registerTempTable

In [76]:
#register temp table
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3),('Alice','Bob',0.4)],
                       ['from','to','amt'])
x.registerTempTable('transactions')
y=sqlc.sql('SELECT * FROM TRANSACTIONS WHERE amt>0.1')
x.show()
y.show()

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
|Alice|  Bob|0.4|
+-----+-----+---+

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
| fred|  kev|0.2|
| nico|david|0.3|
|Alice|  Bob|0.4|
+-----+-----+---+



## replace

In [77]:
#replace
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3),('Alice','Bob',0.4)],
                       ['from','to','amt'])
y=x.replace('Alice','Gabi',['from','to'])
x.show()
y.show()

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
|Alice|  Bob|0.4|
+-----+-----+---+

+----+-----+---+
|from|   to|amt|
+----+-----+---+
|Gabi|  Bob|0.1|
|fred|  kev|0.2|
|nico|david|0.3|
|Gabi|  Bob|0.4|
+----+-----+---+



## rollup

In [81]:
#rollup
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3),('Alice','Bob',0.4)],
                       ['from','to','amt'])
y=x.rollup(['from']) #grouped data
x.show()
print(y)
#aggregations can be performed on numeric columns
y.sum().show()
y.avg().show()

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
|Alice|  Bob|0.4|
+-----+-----+---+

+-----+--------+
| from|sum(amt)|
+-----+--------+
|Alice|     0.5|
| fred|     0.2|
| nico|     0.3|
| null|     1.0|
+-----+--------+

+-----+--------+
| from|avg(amt)|
+-----+--------+
|Alice|    0.25|
| fred|     0.2|
| nico|     0.3|
| null|    0.25|
+-----+--------+



## sample

In [87]:
#sample
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3),('Alice','Bob',0.4)],
                       ['from','to','amt'])
y=x.sample(True,0.5) #with replacement=True,fraction of dataframe
x.show()
y.show()

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
|Alice|  Bob|0.4|
+-----+-----+---+

+-----+---+---+
| from| to|amt|
+-----+---+---+
|Alice|Bob|0.1|
|Alice|Bob|0.4|
+-----+---+---+



## sampleBy

In [93]:
#sampleBy
x=sqlc.createDataFrame([('fred','Bob',0.1),('fred','kev',0.2),('nico','david',0.3),('nico','Bob',0.4)],
                       ['from','to','amt'])
y=x.sampleBy('from',fractions={'fred':0.8,'nico':0.2}) #cols,fractions to divide based on col values
x.show()
y.show()

+----+-----+---+
|from|   to|amt|
+----+-----+---+
|fred|  Bob|0.1|
|fred|  kev|0.2|
|nico|david|0.3|
|nico|  Bob|0.4|
+----+-----+---+

+----+-----+---+
|from|   to|amt|
+----+-----+---+
|fred|  Bob|0.1|
|fred|  kev|0.2|
|nico|david|0.3|
+----+-----+---+



## schema

In [94]:
#schema
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3),('Alice','Bob',0.4)],
                       ['from','to','amt'])
y=x.schema #gives schema of the dataframe
x.show()
print(y)

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
|Alice|  Bob|0.4|
+-----+-----+---+

StructType(List(StructField(from,StringType,true),StructField(to,StringType,true),StructField(amt,DoubleType,true)))


## select

In [95]:
#select
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3),('Alice','Bob',0.4)],
                       ['from','to','amt'])
y=x.select(['from','to']) #select required cols
x.show()
y.show()

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
|Alice|  Bob|0.4|
+-----+-----+---+

+-----+-----+
| from|   to|
+-----+-----+
|Alice|  Bob|
| fred|  kev|
| nico|david|
|Alice|  Bob|
+-----+-----+



## selectExpr

In [106]:
#selectExpr
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3),('Alice','Bob',0.4)],
                       ['from','to','amt'])
y=x.selectExpr(['substr(from,2)','amt+10']) #substring of values in 'from' col staring from 2nd letter to end
x.show()
y.show()

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
|Alice|  Bob|0.4|
+-----+-----+---+

+------------------------------+----------+
|substring(from, 2, 2147483647)|(amt + 10)|
+------------------------------+----------+
|                          lice|      10.1|
|                           red|      10.2|
|                           ico|      10.3|
|                          lice|      10.4|
+------------------------------+----------+



In [108]:
#selectExpr
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3),('Alice','Bob',0.4)],
                       ['from','to','amt'])
y=x.selectExpr(['substr(from,1,2)','amt+10']) #substring of values in 'from' col starting from 1st letter 
                                                        # considering 2 letters as substring 
x.show()
y.show()

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
|Alice|  Bob|0.4|
+-----+-----+---+

+---------------------+----------+
|substring(from, 1, 2)|(amt + 10)|
+---------------------+----------+
|                   Al|      10.1|
|                   fr|      10.2|
|                   ni|      10.3|
|                   Al|      10.4|
+---------------------+----------+



## show

In [109]:
#show
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3),('Alice','Bob',0.4)],
                       ['from','to','amt'])
x.show() #output returned to object driver

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
|Alice|  Bob|0.4|
+-----+-----+---+



## sort

In [111]:
#sort
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3),('Alice','Bob',0.4)],
                       ['from','to','amt'])
y=x.sort('amt',ascending=False)
x.show()
y.show()

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
|Alice|  Bob|0.4|
+-----+-----+---+

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.4|
| nico|david|0.3|
| fred|  kev|0.2|
|Alice|  Bob|0.1|
+-----+-----+---+



## sortWithinPartitions

In [126]:
#sortWithinPartitions
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3),('Alice','Bob',0.4)],
                       ['from','to','amt'])
x=x.repartition(2)
y=x.sortWithinPartitions(['amt'])
x.show()
print(x.rdd.glom().collect()) #glom() adds the rdds as tuples for each partition
print('====================')
print(y.rdd.glom().collect())

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.4|
| nico|david|0.3|
|Alice|  Bob|0.1|
| fred|  kev|0.2|
+-----+-----+---+

[[Row(from='Alice', to='Bob', amt=0.4), Row(from='nico', to='david', amt=0.3)], [Row(from='Alice', to='Bob', amt=0.1), Row(from='fred', to='kev', amt=0.2)]]
[[Row(from='nico', to='david', amt=0.3), Row(from='Alice', to='Bob', amt=0.4)], [Row(from='Alice', to='Bob', amt=0.1), Row(from='fred', to='kev', amt=0.2)]]


## stat

In [131]:
#stat
x=sqlc.createDataFrame([('Alice','Bob',0.1,12),('fred','kev',0.2,20),('nico','david',0.3,33),('Alice','Bob',0.4,40)],
                       ['from','to','amt','fee'])
y=x.stat #can perform func on y
x.show()
print(y)
print('---------------')
print(y.cov('amt','fee'))
y.corr('amt','fee')


+-----+-----+---+---+
| from|   to|amt|fee|
+-----+-----+---+---+
|Alice|  Bob|0.1| 12|
| fred|  kev|0.2| 20|
| nico|david|0.3| 33|
|Alice|  Bob|0.4| 40|
+-----+-----+---+---+

---------------
1.6166666666666665


0.9933707902922091

## subtract

In [132]:
#subtract
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3),('Alice','Bob',0.4)],
                       ['from','to','amt'])
y=sqlc.createDataFrame([('Alice','Bob',0.5),('fred','kev',0.6),('nico','david',0.3),('Alice','Bob',0.4)],
                       ['from','to','amt'])
z=x.subtract(y)
x.show()
y.show()
z.show()

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
|Alice|  Bob|0.4|
+-----+-----+---+

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.5|
| fred|  kev|0.6|
| nico|david|0.3|
|Alice|  Bob|0.4|
+-----+-----+---+

+-----+---+---+
| from| to|amt|
+-----+---+---+
|Alice|Bob|0.1|
| fred|kev|0.2|
+-----+---+---+



## take

In [134]:
#take
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3),('Alice','Bob',0.4)],
                       ['from','to','amt'])
y=x.take(2) #list object returned to driver
x.show()
print(y) 

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
|Alice|  Bob|0.4|
+-----+-----+---+

[Row(from='Alice', to='Bob', amt=0.1), Row(from='fred', to='kev', amt=0.2)]


## toDF

In [136]:
#toDF
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3),('Alice','Bob',0.4)],
                       ['from','to','amt'])
y=x.toDF('seller','buyer','cost')
x.show()
y.show()

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
|Alice|  Bob|0.4|
+-----+-----+---+

+------+-----+----+
|seller|buyer|cost|
+------+-----+----+
| Alice|  Bob| 0.1|
|  fred|  kev| 0.2|
|  nico|david| 0.3|
| Alice|  Bob| 0.4|
+------+-----+----+



## toJSON

In [139]:
#toJSON
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3),('Alice','Bob',0.4)],
                       ['from','to','amt'])
y=x.toJSON()
x.show()
print(y)
print('-----------------')
print(y.collect())

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
|Alice|  Bob|0.4|
+-----+-----+---+

MapPartitionsRDD[1785] at toJavaRDD at NativeMethodAccessorImpl.java:0
-----------------
['{"from":"Alice","to":"Bob","amt":0.1}', '{"from":"fred","to":"kev","amt":0.2}', '{"from":"nico","to":"david","amt":0.3}', '{"from":"Alice","to":"Bob","amt":0.4}']


## toPandas

In [142]:
#toPandas
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3),('Alice','Bob',0.4)],
                       ['from','to','amt'])
y=x.toPandas() #pandas obj returned to driver
x.show()
print(type(y))
y.head()

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
|Alice|  Bob|0.4|
+-----+-----+---+

<class 'pandas.core.frame.DataFrame'>


,from,to,amt
0,Alice,Bob,0.1
1,fred,kev,0.2
2,nico,david,0.3
3,Alice,Bob,0.4


## unionAll

In [146]:
#unionAll
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3),('Alice','Bob',0.4)],
                       ['from','to','amt'])
y=sqlc.createDataFrame([('Alice','Bob',0.1),('kun','kev',0.2),('nico','david',0.3),('Alice','kun',0.4)],
                       ['from','to','amt'])
z=x.unionAll(y)
x.show()
y.show()
z.show()
z.dropDuplicates().show()

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
|Alice|  Bob|0.4|
+-----+-----+---+

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
|  kun|  kev|0.2|
| nico|david|0.3|
|Alice|  kun|0.4|
+-----+-----+---+

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
|Alice|  Bob|0.4|
|Alice|  Bob|0.1|
|  kun|  kev|0.2|
| nico|david|0.3|
|Alice|  kun|0.4|
+-----+-----+---+

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.4|
| nico|david|0.3|
|Alice|  Bob|0.1|
|  kun|  kev|0.2|
|Alice|  kun|0.4|
| fred|  kev|0.2|
+-----+-----+---+



## unpersist

In [151]:
#unpersist
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3),('Alice','Bob',0.4)],
                       ['from','to','amt'])
x.cache()#loaded into memory
x.count()
x.show()

print(x.is_cached)
x.unpersist() #unpersisted from cache
print(x.is_cached)

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
|Alice|  Bob|0.4|
+-----+-----+---+

True
False


## where

In [152]:
#where(filter)
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3),('Alice','Bob',0.4)],
                       ['from','to','amt'])
y=x.where('amt>0.2')
x.show()
y.show()

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
|Alice|  Bob|0.4|
+-----+-----+---+

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
| nico|david|0.3|
|Alice|  Bob|0.4|
+-----+-----+---+



## withColumn

In [154]:
#withColumn
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',None),('Alice','Bob',0.4)],
                       ['from','to','amt'])
y=x.withColumn('conf',x.amt.isNotNull())
x.show()
y.show()

+-----+-----+----+
| from|   to| amt|
+-----+-----+----+
|Alice|  Bob| 0.1|
| fred|  kev| 0.2|
| nico|david|null|
|Alice|  Bob| 0.4|
+-----+-----+----+

+-----+-----+----+-----+
| from|   to| amt| conf|
+-----+-----+----+-----+
|Alice|  Bob| 0.1| true|
| fred|  kev| 0.2| true|
| nico|david|null|false|
|Alice|  Bob| 0.4| true|
+-----+-----+----+-----+



## withColumnRenamed

In [156]:
#withColumnRenamed
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3),('Alice','Bob',0.4)],
                       ['from','to','amt'])
y=x.withColumnRenamed('amt','amount')
x.show()
y.show()

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
|Alice|  Bob|0.4|
+-----+-----+---+

+-----+-----+------+
| from|   to|amount|
+-----+-----+------+
|Alice|  Bob|   0.1|
| fred|  kev|   0.2|
| nico|david|   0.3|
|Alice|  Bob|   0.4|
+-----+-----+------+



## write

In [160]:
#write
x=sqlc.createDataFrame([('Alice','Bob',0.1),('fred','kev',0.2),('nico','david',0.3),('Alice','Bob',0.4)],
                       ['from','to','amt'])
y=x.write.mode('overwrite').json('dataframeWriteTextExample.json')
x.show()
sqlc.read.json('dataframeWriteTextExample.json').show()

+-----+-----+---+
| from|   to|amt|
+-----+-----+---+
|Alice|  Bob|0.1|
| fred|  kev|0.2|
| nico|david|0.3|
|Alice|  Bob|0.4|
+-----+-----+---+

+---+-----+-----+
|amt| from|   to|
+---+-----+-----+
|0.1|Alice|  Bob|
|0.2| fred|  kev|
|0.3| nico|david|
|0.4|Alice|  Bob|
+---+-----+-----+

